In [7]:
from flask import Flask
from datetime import date, timedelta
import yfinance as yf #Alternative package if webreader does not work: pip install yfinance
import joblib
import numpy as np # Fundamental package for scientific computing with Python
import pandas as pd # Additional functions for analysing and manipulating data
from ibapi.client import *
from ibapi.wrapper import *
from ibapi.contract import Contract
import threading
import time

In [8]:
class IBapi(EWrapper, EClient):
    def __init__(self):
        EClient.__init__(self, self)
        self.data = [] #Initialize variable to store candle

    def historicalData(self, reqId, bar):
        print(f'DateTime: {bar.date}  Open: {bar.open}  High: {bar.high} Close: {bar.close}  Low: {bar.low}  Volume: {bar.volume}')
        self.data.append([bar.date, bar.open,bar.high,bar.close,bar.low,bar.volume])

    def cleanData(self):
        print('Cleaning Data')
        self.data.clear()
    
def run_loop():
    app.run()

In [9]:
app = IBapi()
# app.disconnect()
app.connect('127.0.0.1', 7497, 123)

#Start the socket in a thread
api_thread = threading.Thread(target=run_loop, daemon=True)
api_thread.start()
time.sleep(7)

ERROR -1 2104 Connessione con il Server dei dati di mercato è OK:hfarm
ERROR -1 2104 Connessione con il Server dei dati di mercato è OK:usfarm.nj
ERROR -1 2104 Connessione con il Server dei dati di mercato è OK:jfarm
ERROR -1 2104 Connessione con il Server dei dati di mercato è OK:usfuture
ERROR -1 2104 Connessione con il Server dei dati di mercato è OK:cashfarm
ERROR -1 2104 Connessione con il Server dei dati di mercato è OK:eufarmnj
ERROR -1 2104 Connessione con il Server dei dati di mercato è OK:usfarm
ERROR -1 2106 Connessione con il HMDS data farm è OK:euhmds
ERROR -1 2106 Connessione con il HMDS data farm è OK:fundfarm
ERROR -1 2106 Connessione con il HMDS data farm è OK:ushmds
ERROR -1 2158 Connessione alla raccolta dati Sec-def stabilita:secdefil


In [10]:
def create_contract(symbol, secType='IND', exchange='CBOE', currency='USD'):
    contract = Contract()
    contract.symbol = symbol
    contract.secType = secType
    contract.exchange = exchange
    contract.currency = currency
    return contract

def request_historical_data(app, contract):
    app.cleanData()
    app.reqHistoricalData(1, contract, '', "2 D", "1 Day", "TRADES", 0, 1, False, [])
    time.sleep(3)
    return app.data

def process_data(data, result_df, prefix):
    if len(data) == 0:
        print('ERROR - Probabilmente ci sono problemi di connessione alla TWS')
        return result_df

    df = pd.DataFrame(data, columns=['Date', 'Open', 'High', 'Close', 'Low', 'Volume'])
    df['Date'] = df['Date'].str[:17]
    df['Date'] = df['Date'].str[:4] + '-' + df['Date'].str[4:6] + '-' + df['Date'].str[6:8] + '' + df['Date'].str[8:]
    df['Date'] = pd.to_datetime(df['Date'], format="%Y-%m-%d")

    result_df[f'feat_{prefix}Open'] = df['Open']
    result_df[f'feat_{prefix}PrevClose'] = df['Close'].shift(1)

    return result_df

In [11]:
result_df = pd.DataFrame()

# VVIX
vvix_contract = create_contract('VVIX')
vvix_data = request_historical_data(app, vvix_contract)
result_df = process_data(vvix_data, result_df, 'vvix')

# VIX
vix_contract = create_contract('VIX')
vix_data = request_historical_data(app, vix_contract)
result_df = process_data(vix_data, result_df, 'vix')

# SPX
spx_contract = create_contract('SPX')
spx_data = request_historical_data(app, spx_contract)

if len(spx_data) == 0:
    print('ERROR - Probabilmente ci sono problemi di connessione alla TWS')
else:
    spx = pd.DataFrame(spx_data, columns=['Date', 'Open', 'High', 'Close', 'Low', 'Volume'])
    spx['Date'] = spx['Date'].str[:17]
    spx['Date'] = spx['Date'].str[:4] + '-' + spx['Date'].str[4:6] + '-' + spx['Date'].str[6:8] + '' + spx['Date'].str[8:]
    spx['Date'] = pd.to_datetime(spx['Date'], format="%Y-%m-%d")

    result_df['feat_Open-PrevClose'] = np.where(spx['Open'] - spx['Close'].shift(1) > 0, 1, 0)
    result_df['Date'] = pd.to_datetime(spx['Date'])

# Print and return result
print('Dati inseriti nel DF', result_df)


Cleaning Data
DateTime: 20240801  Open: 94.0  High: 117.24 Close: 111.18  Low: 93.31  Volume: 0
DateTime: 20240802  Open: 119.84  High: 162.68 Close: 136.81  Low: 119.03  Volume: 0
Cleaning Data
DateTime: 20240802  Open: 20.52  High: 29.66 Close: 23.39  Low: 20.01  Volume: 0
DateTime: 20240805  Open: 23.39  High: 41.8 Close: 34.22  Low: 23.39  Volume: 0
Cleaning Data
DateTime: 20240801  Open: 5537.84  High: 5566.16 Close: 5446.68  Low: 5410.42  Volume: 0
DateTime: 20240802  Open: 5376.63  High: 5383.89 Close: 5346.56  Low: 5302.03  Volume: 0
Dati inseriti nel DF    feat_vvixOpen  feat_vvixPrevClose  feat_vixOpen  feat_vixPrevClose  \
0          94.00                 NaN         20.52                NaN   
1         119.84              111.18         23.39              23.39   

   feat_Open-PrevClose       Date  
0                    0 2024-08-01  
1                    0 2024-08-02  


In [4]:
resultDataFrame = pd.DataFrame()

orderType = 'noTrade'

time.sleep(3) #Sleep interval to allow time for connection to server

#Create contract object
contract = Contract()
contract.symbol = 'VVIX'
contract.secType = 'IND'
contract.exchange = 'CBOE'
contract.currency = 'USD'

#Request historical candles
app.reqHistoricalData(1, contract, '', "2 D", "1 Day", "TRADES", 0, 1, False, [])

time.sleep(3) #sleep to allow enough time for data to be returned
# app.disconnect()

print('Dati ottenuti da TWS', app.data)

if(len(app.data)==0):
    print('ERROR - Probabilmente ci sono problemi di connessione alla TWS')
else:
    vvix = pd.DataFrame(app.data, columns=['Date','Open', 'High', 'Close', 'Low', 'Volume'])
    vvix['Date'] = vvix['Date'].str[:17]
    vvix['Date'] = vvix['Date'].str[:4] + '-' + vvix['Date'].str[4:6] +  '-' + vvix['Date'].str[6:8] + '' + vvix['Date'].str[8:]
    
    vvix['Date'] = pd.to_datetime(vvix['Date'], format="%Y-%m-%d")# %H:%M:%S")
    
    # setting first name as index column

    
    resultDataFrame['feat_vvixOpen'] = vvix['Open']
    resultDataFrame['feat_vvixPrevClose'] = vvix['Close'].shift(1)
    # vvix = vvix.drop(['Open','Close', 'High','Low', 'Volume'], axis=1)

    print('Dati inseriti nel DF', vvix)

    # -------------------------

#Create contract object
contract = Contract()
contract.symbol = 'VIX'
contract.secType = 'IND'
contract.exchange = 'CBOE'
contract.currency = 'USD'


#Request historical candles
app.cleanData()
app.reqHistoricalData(1, contract, '', "2 D", "1 Day", "TRADES", 0, 1, False, [])
time.sleep(3)

print('Dati ottenuti da TWS', app.data)

# app.connect('127.0.0.1', 7497, 123)

#Start the socket in a thread
# api_thread = threading.Thread(target=run_loop, daemon=True)
# api_thread.start()
if(len(app.data)==0):
    print('ERROR - Probabilmente ci sono problemi di connessione alla TWS')
else:
    vix = pd.DataFrame(app.data, columns=['Date','Open', 'High', 'Close', 'Low', 'Volume'])
    vix['Date'] = vix['Date'].str[:17]
    vix['Date'] = vix['Date'].str[:4] + '-' + vix['Date'].str[4:6] +  '-' + vix['Date'].str[6:8] + '' + vix['Date'].str[8:]
    
    vix['Date'] = pd.to_datetime(vix['Date'], format="%Y-%m-%d")# %H:%M:%S")
    
    resultDataFrame['feat_vixOpen'] = vix['Open']
    resultDataFrame['feat_vixPrevClose'] = vix['Close'].shift(1)

    # vix = vix.drop(['Open','Close', 'High','Low', 'Volume'], axis=1)
    
    # Seleziona le colonne con il prefisso "feat"
    feat_cols = [col for col in resultDataFrame.columns if 'feat' in col]
    # resultDataFrame.dropna(how='any', inplace=True)

#Create contract object
contract = Contract()
contract.symbol = 'SPX'
contract.secType = 'IND'
contract.exchange = 'CBOE'
contract.currency = 'USD'


#Request historical candles
app.cleanData()
app.reqHistoricalData(1, contract, '', "2 D", "1 Day", "TRADES", 0, 1, False, [])
time.sleep(3)

print('Dati ottenuti da TWS', app.data)

# app.connect('127.0.0.1', 7497, 123)

#Start the socket in a thread
# api_thread = threading.Thread(target=run_loop, daemon=True)
# api_thread.start()
if(len(app.data)==0):
    print('ERROR - Probabilmente ci sono problemi di connessione alla TWS')
else:
    spx = pd.DataFrame(app.data, columns=['Date','Open', 'High', 'Close', 'Low', 'Volume'])
    spx['Date'] = spx['Date'].str[:17]
    spx['Date'] = spx['Date'].str[:4] + '-' + spx['Date'].str[4:6] +  '-' + spx['Date'].str[6:8] + '' + spx['Date'].str[8:]
    
    spx['Date'] = pd.to_datetime(spx['Date'], format="%Y-%m-%d")# %H:%M:%S")
    
    resultDataFrame['feat_Open-PrevClose'] = np.where(spx['Open']-spx['Close'].shift(1)>0,1,0)
    resultDataFrame['Date'] = pd.to_datetime(spx['Date'])
    # resultDataFrame.set_index("Date", inplace = True)
    # vix = vix.drop(['Open','Close', 'High','Low', 'Volume'], axis=1)
    
    # Seleziona le colonne con il prefisso "feat"
    feat_cols = [col for col in resultDataFrame.columns if 'feat' in col]
    # resultDataFrame.dropna(how='any', inplace=True)    



ERROR 1 504 Not connected
ERROR 1 504 Not connected


Dati ottenuti da TWS []
ERROR - Probabilmente ci sono problemi di connessione alla TWS
Cleaning Data


ERROR 1 504 Not connected


Dati ottenuti da TWS []
ERROR - Probabilmente ci sono problemi di connessione alla TWS
Cleaning Data
Dati ottenuti da TWS []
ERROR - Probabilmente ci sono problemi di connessione alla TWS


In [12]:
print('final DF', result_df)
time.sleep(3) #sleep to allow enough time for data to be returned
app.disconnect()
time.sleep(3) #sleep to allow enough time for data to be returned

final DF Empty DataFrame
Columns: []
Index: []


In [13]:
# CLASSIFICATION
result_df = result_df.dropna()
clf = loaded_rf = joblib.load("./BigShort.joblib")
features = [ 'feat_Open-PrevClose', 'feat_vixOpen', 'feat_vixPrevClose', 'feat_vvixOpen', 'feat_vvixPrevClose']

# TEST
# datas = [[date.today(), 1, 25.61000061, 25.43000031, 89.37000275, 79.58999634]] # SELL
# datas = [[date.today(), 0, 23.27000046, 23.79000092, 79.58999634, 79.51000214]] # BUY
# resultDataFrame = pd.DataFrame(datas, columns=['Date','feat_Open-PrevClose', 'feat_vixOpen', 'feat_vixPrevClose', 'feat_vvixOpen', 'feat_vvixPrevClose'])

x_live = result_df[features]

y_live_pred = clf.predict(x_live)

print(y_live_pred)

print(result_df)
if(result_df['Date'].iloc[-1]==(date.today())):
    orderType = 'HOLD' if y_live_pred==1 else 'SELL'
    print('Direzione Prevista:', orderType)
else:
    orderType = 'noTrade'
    print('Dati per oggi non ancora disponibili. Ultima data disponibile:', result_df['Date'].iloc[-1].date())
        
print('Direzione prevista per oggi:', orderType)
    # return orderType

[-1]
   feat_vvixOpen  feat_vvixPrevClose  feat_vixOpen  feat_vixPrevClose  \
1         119.84              111.18         23.39              23.39   

   feat_Open-PrevClose       Date  
1                    0 2024-08-02  
Dati per oggi non ancora disponibili. Ultima data disponibile: 2024-08-02
Direzione prevista per oggi: noTrade
